## 复制列表（或多数内置的可变集合,浅复制）

In [1]:
l1 = [3, 17, [22, 9, 13], (41, 5, 37)]
# 最简单的方式是使用内置的类型构造方法
l2 = list(l1)
print(l2)
print(l2 == l1)
print(l2 is l1)

[3, 17, [22, 9, 13], (41, 5, 37)]
True
False


In [2]:
# 或者用一个更简洁的方式
l3 = l1[:]
print(l3)
print(l3 == l1)
print(l3 is l1)

[3, 17, [22, 9, 13], (41, 5, 37)]
True
False


>`==` 运算符比较两个对象的值（对象中保存的数据），而 `is` 比较对象的标识, 相当于其指向对象的内存地址<p>
>可以看到, `l3`,`l2` `==` `l1`是 `True`, 而 `is` 判断为 `False`, 这是因为构造方法或 `[:]` 做的是浅复制（即复制了最外层容器，副本中的元素是源容器中元素的引用）, 换句话说浅拷贝产生的对象本身是新的，但是它的内容不是新的，只是对原对象的一个引用

### Python对于不可变对象的特殊处理
>上面例子可以看到对于列表这种可变对象来说, 不管是 `list(l1)`这样的内置构造方法还是更简单的`l1[:]`方式, 利用`is`方法可以判断其复制出来的对象其实是不一样的, 因为其内存地址并不一样<p>
>但是对于不可变对象来说, 就不一样了, 看下面例子:

In [3]:
t1 = (1, 2, 3)
t2 = t1[:]
t3 = tuple(t1)
id(t1), id(t2), id(t3)

(4373193640, 4373193640, 4373193640)

>可以看到, 对于元组这种不可变对象来说, 返回的其实是同一个对象的引用, `t1`, `t2`, `t3`都是指的同一个对象, 在`Python`中, 还有`str`, `bytes`, `frozenset`也有这种行为<p>
>我觉得这里可以类比 `Java`中的基本类型, 比如定义`String`, `int`等类型时, 它会先在 `JVM` 中查找是否存在这个字面量,如果存在的话就是直接引用这个, 没有则会创建一块新地址存储

## 为任意对象做深复制和浅复制

>深复制（即副本不共享内部对象的引用）<p>
>就是在另一块地址中创建一个新的变量或容器，同时容器内的元素的地址也是新开辟的，仅仅是值相同而已，是完全的副本

>copy模块提供的 deepcopy 和 copy 函数能为任意对象做深复制和浅复制。

为了演示 `copy()` 和 `deepcopy()` 的用法，定义了一个简单的类， `Bus`。这个类表示运载乘客的校车，在途中乘客会上车或下车, 相当于一个对象中的属性值变化

In [4]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [5]:
import copy
bus1 = Bus(['A1', 'B1', 'C1', 'D1'])
bus2 = copy.copy(bus1)     # 浅复制
bus3 = copy.deepcopy(bus1) # 深复制
id(bus1), id(bus2), id(bus3)

(4371626136, 4371626808, 4371627032)

In [6]:
bus1.drop('B1')
bus2.passengers

['A1', 'C1', 'D1']

In [7]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4372371848, 4372371848, 4371684552)

In [8]:
bus3.passengers

['A1', 'B1', 'C1', 'D1']

👆上面示例代码的说明:
1. 使用 copy 和 deepcopy，创建 3 个不同的 Bus 实例。
2. bus1 中的 'B1' 下车后， bus2 中也没有他了。
3. 审查 passengers 属性后发现， bus1 和 bus2 共享同一个列表对象，因为 bus2 是 bus1 的浅复制副本
4. bus3 是 bus1 的深复制副本，因此它的 passengers 属性指代另一个列表

再看 `bus1`, `bus2`, `bus3`三个对象实例的关系
![](images/bus.png)
可以看到 `bus1` 的浅复制 `bus2` 两个引用同一个对象, 而深复制 `bus3`却是另外一个

### 思考🤔: Python中的浅复制和深复制好像可以类比于Java的浅拷贝和深拷贝
>1、浅拷贝：对基本数据类型进行值传递，对引用数据类型进行引用传递般的拷贝，此为浅拷贝<p>
>2、深拷贝：对基本数据类型进行值传递，对引用数据类型，创建一个新的对象，并复制其内容，此为深拷贝